In [1]:
import pandas as pd
import numpy as np

In [20]:
dataset_ftse_mib = pd.read_csv("./Datasets/FTSEMIB.MI.csv")

In [35]:
len(dataset_ftse_mib)

1268

In [2]:
def get_stocks_by_week(df, week):
    """
    Returns a dataframe of stocks for a given week
    """
    return df[df['Week Number'] == week]

In [15]:
def get_portfolio_weekly_return(portfolio):
  totalReturn = 0
  
  for index, row in portfolio.iterrows():
    log_ret = row["Stock Weekly Return"]
    totalReturn+=log_ret
  return totalReturn/len(portfolio)


In [17]:
def get_ftse_mib_weekly_return(week_number):
  ftse_mib = pd.read_csv("./Datasets/FTSEMIB.MI.csv")
  prev_week_ret = ftse_mib.iloc[180+week_number-7]["log_ret"]
  next_week_ret = ftse_mib.iloc[180+week_number]["log_ret"]
  return next_week_ret-prev_week_ret


In [10]:
test = pd.read_csv("./results.csv")

In [11]:
res = get_stocks_by_week(test, 0)

sorted_df = res.sort_values(by="Beta", ascending=False)
sorted_df.head(20)


,Unnamed: 0,Stock Symbol,Week Number,Alpha,Beta,R-Squared,Res Std. Error,Stock Weekly Return
5068,5068,STLA.MI,0,2.795446e-03,1.822867,0.501008,0.041383,-0.008591
2184,2184,EXO.MI,0,5.355470e-04,1.566642,0.624995,0.030567,0.005022
624,624,BAMI.MI,0,8.926281e-05,1.360341,0.281186,0.023047,0.008328
5224,5224,STM.MI,0,6.067704e-04,1.357758,0.283644,0.022959,-0.000041
5848,5848,UCG.MI,0,-4.747361e-05,1.329340,0.448922,0.022008,-0.006662
1092,1092,BPE.MI,0,-8.900354e-05,1.290830,0.241391,0.020751,0.020998
3432,3432,LDO.MI,0,-3.591453e-03,1.193388,0.153714,0.017737,0.009069
2340,2340,FBK.MI,0,1.319193e-03,1.186950,0.456557,0.017546,0.006053
1248,1248,BZU.MI,0,-8.097588e-04,1.180497,0.378496,0.017356,-0.025273
1404,1404,CNHI.MI,0,3.586125e-04,1.159120,0.413090,0.016733,-0.018219


Test: take top 15 and bottom 15 based on beta

In [12]:
top_10 = sorted_df.head(10)
bottom_10 = sorted_df.tail(10)

In [13]:
frames = [sorted_df.head(10),  sorted_df.tail(10)]
result = pd.concat(frames)


In [16]:
get_portfolio_weekly_return(result)


-0.0038107632750175105

In [18]:
get_ftse_mib_weekly_return(0)

-0.008566399982807801

In [37]:
final_dataframe = pd.DataFrame(columns=["Week","stockSymbols","portfolio Weekly Returns","Ftse Mib weekly returns"])

In [36]:
for weekNumber in range(0,len(dataset_ftse_mib),7):
  print(weekNumber)

0
7
14
21
28
35
42
49
56
63
70
77
84
91
98
105
112
119
126
133
140
147
154
161
168
175
182
189
196
203
210
217
224
231
238
245
252
259
266
273
280
287
294
301
308
315
322
329
336
343
350
357
364
371
378
385
392
399
406
413
420
427
434
441
448
455
462
469
476
483
490
497
504
511
518
525
532
539
546
553
560
567
574
581
588
595
602
609
616
623
630
637
644
651
658
665
672
679
686
693
700
707
714
721
728
735
742
749
756
763
770
777
784
791
798
805
812
819
826
833
840
847
854
861
868
875
882
889
896
903
910
917
924
931
938
945
952
959
966
973
980
987
994
1001
1008
1015
1022
1029
1036
1043
1050
1057
1064
1071
1078
1085
1092
1099
1106
1113
1120
1127
1134
1141
1148
1155
1162
1169
1176
1183
1190
1197
1204
1211
1218
1225
1232
1239
1246
1253
1260
1267


In [38]:
results = pd.read_csv("./results.csv")
for weekNumber in range(0,len(dataset_ftse_mib),7):
  res = get_stocks_by_week(test, weekNumber)
  sorted_df = res.sort_values(by="Beta", ascending=False)
  frames = [sorted_df.head(10),  sorted_df.tail(10)]
  top_bottom_10 = pd.concat(frames)
  if len(top_bottom_10) > 0:
    portfolio_return = get_portfolio_weekly_return(top_bottom_10)
    ftse_mib_return = get_ftse_mib_weekly_return(weekNumber)
    final_dataframe.loc[len(final_dataframe)] = [
        weekNumber, top_bottom_10["Stock Symbol"].values, portfolio_return, ftse_mib_return]


In [39]:
final_dataframe[final_dataframe["Week"] == 0]

,Week,stockSymbols,portfolio Weekly Returns,Ftse Mib weekly returns
0,0,"[STLA.MI, EXO.MI, BAMI.MI, STM.MI, UCG.MI, BPE...",-0.003811,-0.008566


In [25]:
import plotly.express as px


In [40]:
fig = px.line(final_dataframe, x="Week", y=["portfolio Weekly Returns", "Ftse Mib weekly returns"],
              title='Returns')
fig.show()